# Python in Pratica - Lezione 4

## Analisi Dati E-Commerce e Large Language Models

**Data:** 31 gennaio 2026  
**FabLab Bergamo**

---

## 📚 Contenuti di oggi
- **Ripasso**: Pandas e Matplotlib dalle lezioni precedenti
- **Analisi dati reali**: Dataset e-commerce con vendite storiche
- **Visualizzazioni avanzate**: Grafici di spesa per anno e tendenze
- **Large Language Models**: Classificazione automatica di prodotti
- **Integrazione AI**: Usare API OpenAI per analisi di testo
- **Progetto finale**: Dashboard di analisi vendite con categorizzazione AI

# 🔄 RIPASSO LEZIONI PRECEDENTI (15 min)

Cosa abbiamo imparato nelle lezioni precedenti? Rivediamo velocemente i concetti che useremo oggi.

## Pandas: il nostro strumento per i dati
- **DataFrame**: tabelle di dati come Excel, ma più potenti
- **Caricamento CSV**: `pd.read_csv()`
- **Filtraggio**: `df[df['colonna'] > valore]`
- **Raggruppamento**: `df.groupby('colonna').agg()`
- **Statistiche**: `.mean()`, `.sum()`, `.count()`

## Matplotlib: visualizzazioni
- **Grafici a barre**: `df.plot(kind='bar')`
- **Grafici a linee**: `df.plot()`
- **Personalizzazione**: titoli, colori, dimensioni

In [ ]:
# Installazione librerie per questa lezione
!pip install pandas matplotlib openai requests

import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from datetime import datetime
import numpy as np

print("✅ Librerie installate e importate con successo!")

# 📊 ANALISI DATI E-COMMERCE (30 min)

Oggi analizzeremo dati reali di vendite online! Useremo un dataset che contiene:
- **Ordini storici** di un negozio online
- **Informazioni prodotti**: nome, prezzo, quantità
- **Date** delle vendite
- **Dati clienti**: per capire i comportamenti d'acquisto

## 📤 Caricamento del Dataset

Per caricare il file CSV in Google Colab:
1. Clicca sull'icona **📁 Files** nella barra laterale sinistra  
2. Clicca su **📤 Upload to session storage**
3. Seleziona il file `Retail.OrderHistory.2.csv`
4. Aspetta che il caricamento si completi

⚠️ **Importante**: Assicurati che il file si chiami esattamente `Retail.OrderHistory.2.csv`

In [ ]:
# STEP 1: Caricare i dati di vendita
try:
    # Prova a caricare il file CSV
    df_vendite = pd.read_csv('Retail.OrderHistory.2.csv')
    print("✅ Dataset caricato con successo!")

    # Informazioni di base sul dataset
    print(f"📊 Righe nel dataset: {len(df_vendite):,}")
    print(f"📋 Colonne: {list(df_vendite.columns)}")
    print(f"💾 Dimensioni: {df_vendite.shape}")

except FileNotFoundError:
    print("❌ File non trovato! Assicurati di aver caricato 'Retail.OrderHistory.2.csv'")
    print("📝 Per caricare il file:")
    print("   1. Clicca sull'icona Files (📁) a sinistra")
    print("   2. Clicca 'Upload to session storage'")
    print("   3. Seleziona il file CSV")

    # Creiamo dati di esempio per continuare la lezione
    print("\n🔧 Creando dati di esempio per continuare...")

    # Dati di esempio realistici
    import random
    from datetime import datetime, timedelta

    prodotti_esempio = [
        "iPhone 15", "Samsung Galaxy S24", "MacBook Air", "Dell XPS 13",
        "Sony WH-1000XM5", "AirPods Pro", "iPad Air", "Microsoft Surface",
        "Canon EOS R5", "Nintendo Switch", "PS5 Console", "Xbox Series X",
        "Sneakers Nike", "Adidas Ultraboost", "Levi's Jeans", "Tommy Hilfiger Shirt"
    ]

    # Genera 1000 ordini di esempio
    dati_esempio = []
    for i in range(1000):
        data_ordine = datetime(2022, 1, 1) + timedelta(days=random.randint(0, 730))
        prodotto = random.choice(prodotti_esempio)
        quantita = random.randint(1, 3)
        prezzo_unitario = random.randint(50, 2000)

        dati_esempio.append({
            'OrderDate': data_ordine.strftime('%Y-%m-%d'),
            'ProductName': prodotto,
            'Quantity': quantita,
            'UnitPrice': prezzo_unitario,
            'TotalAmount': quantita * prezzo_unitario
        })

    df_vendite = pd.DataFrame(dati_esempio)
    print("✅ Dataset di esempio creato!")
    print(f"📊 Righe nel dataset: {len(df_vendite):,}")
    print(f"📋 Colonne: {list(df_vendite.columns)}")

In [ ]:
# STEP 2: Esplorare i dati (come nelle lezioni precedenti con i voti!)
print("🔍 ESPLORAZIONE DEL DATASET")
print("=" * 50)

# Prime righe (come facevamo con i dizionari)
print("📋 PRIME 5 RIGHE:")
print(df_vendite.head())

print("\n📊 INFORMAZIONI GENERALI:")
print(f"Periodo dati: {df_vendite['Order Date'].min()} → {df_vendite['Order Date'].max()}")
print(f"Numero prodotti unici: {df_vendite['Product Name'].nunique()}")
print(f"Totale fatturato: €{df_vendite['Total Owed'].sum():,.2f}")
print(f"Ordine medio: €{df_vendite['Total Owed'].mean():.2f}")

In [ ]:
# STEP 3: Preparare i dati per l'analisi temporale
# Convertire la colonna data in formato datetime (come nelle lezioni precedenti)
df_vendite['Order Date'] = pd.to_datetime(df_vendite['Order Date'], format='ISO8601')

# Estrarre anno e mese (nuovo concetto!)
df_vendite['Anno'] = df_vendite['Order Date'].dt.year
df_vendite['Mese'] = df_vendite['Order Date'].dt.month
df_vendite['AnnoMese'] = df_vendite['Order Date'].dt.to_period('M')

print("✅ Date convertite e colonne temporali aggiunte")
print("\n📅 DISTRIBUZIONE PER ANNO:")
print(df_vendite['Anno'].value_counts().sort_index())

# 📈 VISUALIZZAZIONI CON MATPLOTLIB (20 min)

Ora creiamo dei grafici per capire meglio i dati! Useremo Matplotlib come nella lezione precedente, ma con dati più interessanti.

## Analisi delle vendite per anno (come le medie per materia!)

In [ ]:
# GRAFICO 1: Spesa totale per anno (come le medie per materia!)
print("📊 ANALISI VENDITE PER ANNO")
print("=" * 40)

# Raggruppare per anno e calcolare statistiche (come facevamo con i voti!)
vendite_per_anno = df_vendite.groupby('Anno').agg({
    'Total Owed': ['sum', 'count', 'mean']
}).round(2)

# Rinominare le colonne per chiarezza
vendite_per_anno.columns = ['Fatturato_Totale', 'Numero_Ordini', 'Ordine_Medio']

print(vendite_per_anno)

# Creare il grafico
plt.figure(figsize=(12, 5))

# Subplot 1: Fatturato totale
plt.subplot(1, 2, 1)
vendite_per_anno['Fatturato_Totale'].plot(kind='bar', color='steelblue', edgecolor='black')
plt.title('Fatturato Totale per Anno', fontsize=14, fontweight='bold')
plt.xlabel('Anno')
plt.ylabel('Fatturato (€)')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Subplot 2: Numero di ordini
plt.subplot(1, 2, 2)
vendite_per_anno['Numero_Ordini'].plot(kind='bar', color='orange', edgecolor='black')
plt.title('Numero Ordini per Anno', fontsize=14, fontweight='bold')
plt.xlabel('Anno')
plt.ylabel('Numero Ordini')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# GRAFICO 2: Andamento mensile (tendenze nel tempo)
print("\n📈 ANDAMENTO ACQUISTI MENSILI")
print("=" * 40)

# Raggruppare per mese e calcolare fatturato
vendite_mensili = df_vendite.groupby('AnnoMese')['Total Owed'].sum()

print(f"Mesi analizzati: {len(vendite_mensili)}")
print(f"Fatturato medio mensile: €{vendite_mensili.mean():,.2f}")

# Grafico a linee (come nella lezione precedente!)
plt.figure(figsize=(15, 6))
vendite_mensili.plot(marker='o', linewidth=2, color='green')
plt.title('📊 Andamento Vendite Mensili', fontsize=16, fontweight='bold')
plt.xlabel('Mese')
plt.ylabel('Fatturato (€)')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Trova il mese migliore e peggiore
mese_migliore = vendite_mensili.idxmax()
fatturato_migliore = vendite_mensili.max()
mese_peggiore = vendite_mensili.idxmin()
fatturato_peggiore = vendite_mensili.min()

print(f"Mese migliore: {mese_migliore} (€{fatturato_migliore:,.2f})")
print(f"Mese peggiore: {mese_peggiore} (€{fatturato_peggiore:,.2f})")

# 🤖 LARGE LANGUAGE MODELS (30 min)

Ora arriva la parte più interessante! Useremo l'Intelligenza Artificiale per classificare automaticamente i nostri prodotti.

## Cosa sono i Large Language Models?
- **Modelli di AI** addestrati su enormi quantità di testo
- **Capiscono** e **generano** testo in linguaggio naturale  
- **Esempi**: ChatGPT, Claude, Gemini
- **Usi pratici**: classificazione, traduzione, analisi di sentiment

## Il nostro obiettivo
Classificare automaticamente i prodotti in categorie:
- **Elettronica**: iPhone, laptop, cuffie...
- **Abbigliamento**: scarpe, jeans, t-shirt...
- **Casa**: mobili, elettrodomestici...
- **Gaming**: console, videogiochi...

## API che useremo
Per questa lezione useremo **OpenAI API** (ChatGPT), ma esistono alternative gratuite:
- **Ollama** (locale, gratuito)
- **Google Gemini** (API gratuita limitata)
- **Anthropic Claude** (API a pagamento)
- **OpenRouter** (accesso a vari modelli)

In [ ]:
# CONFIGURAZIONE API - AGGIORNATA PER OPENAI V1.0+!
print("🔐 CONFIGURAZIONE API")
print("=" * 30)

# Per usare l'API OpenAI, serve una chiave API
# ATTENZIONE: Non condividere mai la tua chiave API!

# Opzione 1: Inserire manualmente (per il corso)
api_key = input("Inserisci la tua OpenAI API key (o premi ENTER per usare esempio offline): ")

if api_key.strip() == "":
    print("⚠️  Nessuna API key fornita")
    print("🔧 Useremo un classificatore di esempio offline")
    USE_REAL_API = False
    client = None
else:
    print("✅ API key configurata")
    USE_REAL_API = True

    # Importa e configura OpenAI (nuova API v1.0+)
    try:
        from openai import OpenAI
        client = OpenAI(api_key=api_key)
        print("✅ OpenAI configurato (v1.0+)")
    except ImportError:
        print("❌ Libreria OpenAI non trovata")
        print("💡 Esegui: !pip install openai")
        USE_REAL_API = False
        client = None
    except Exception as e:
        print(f"❌ Errore configurazione: {e}")
        USE_REAL_API = False
        client = None

print(f"\n🤖 Modalità API: {'Reale' if USE_REAL_API else 'Simulazione'}")

In [ ]:
# FUNZIONE PER CLASSIFICARE PRODOTTI IN BATCH CON AI (NUOVA VERSIONE!)
def classifica_prodotti_batch(lista_prodotti, batch_size=15):
    """
    Classifica prodotti in batch usando AI v1.0+ con formato JSON
    """

    if USE_REAL_API and client:
        try:
            # Troncare nomi prodotti a 80 caratteri ASCII per evitare overflow token
            prodotti_mapping = {}  # Dizionario per mappare troncati -> originali
            prodotti_troncati = []

            for prod_originale in lista_prodotti:
                # Tronca a 80 caratteri ASCII
                prod_troncato = prod_originale.encode('ascii', errors='ignore').decode('ascii')[:80]
                if len(prod_originale) > 80:
                    prod_troncato = prod_troncato.rstrip() + "..."

                prodotti_troncati.append(prod_troncato)
                prodotti_mapping[prod_troncato] = prod_originale

            prodotti_json = json.dumps(prodotti_troncati, ensure_ascii=False)

            # Prompt per classificazione batch in formato JSON
            system_prompt = """Sei un esperto di e-commerce su Amazon.it. Classifica questi acquisti nelle seguenti categorie per consentire l'analisi della spesa:
            - Elettronica
            - Informatica
            - Libri
            - Casa e Giardino
            - Giochi e Giocattoli
            - Sport e Tempo Libero
            - Abbigliamento
            - Salute e Bellezza
            - Auto e Moto
            - Altro

            Rispondi SOLO con un oggetto JSON nel formato:
            {"prodotto1": "categoria", "prodotto2": "categoria", ...}

            NON aggiungere commenti, spiegazioni o altro testo. Restringi la tua risposta alle categorie elencate."""

            user_prompt = f"Classifica questi prodotti: {prodotti_json}"

            # Richiesta all'API OpenAI v1.0+ con token aumentati
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                max_tokens=3000,
                temperature=0
            )

            # Estrae il contenuto della risposta
            content = response.choices[0].message.content.strip()

            # Debug per monitorare la risposta
            print(f"   Risposta (primi 100 char): {content[:100]}...")

            # Prova a fare il parsing del JSON
            try:
                classificazioni_troncate = json.loads(content)

                # Rimappa alle chiavi originali usando il dizionario di mapping
                classificazioni_finali = {}
                for prod_troncato, categoria in classificazioni_troncate.items():
                    if prod_troncato in prodotti_mapping:
                        prod_originale = prodotti_mapping[prod_troncato]
                        classificazioni_finali[prod_originale] = categoria
                    else:
                        # Fallback: cerca match parziale
                        for orig in lista_prodotti:
                            if orig[:80].startswith(prod_troncato[:50]):
                                classificazioni_finali[orig] = categoria
                                break

                # Assegna "Altro" ai prodotti non classificati
                for prod in lista_prodotti:
                    if prod not in classificazioni_finali:
                        classificazioni_finali[prod] = "Altro"

                return classificazioni_finali

            except json.JSONDecodeError as e:
                print(f"⚠️  Errore parsing JSON: {e}")
                print(f"   Contenuto completo: {content[:200]}...")
                return {prod: "Altro" for prod in lista_prodotti}

        except Exception as e:
            print(f"❌ Errore API: {e}")
            return {prod: "Errore" for prod in lista_prodotti}

    else:
        # Classificatore offline semplice (regole if-else)
        classificazioni = {}
        for prodotto in lista_prodotti:
            nome_lower = prodotto.lower()

            if any(word in nome_lower for word in ['libro', 'romanzo', 'manuale', 'guida', 'fumetto', 'kindle']):
                categoria = "Libri"
            elif any(word in nome_lower for word in ['gioco', 'giocattolo', 'bambino', 'bambini', 'puzzle', 'lego', 'barbie']):
                categoria = "Giochi e Giocattoli"
            elif any(word in nome_lower for word in ['iphone', 'samsung', 'laptop', 'computer', 'cavo', 'caricatore', 'mouse', 'tastiera']):
                categoria = "Elettronica"
            elif any(word in nome_lower for word in ['casa', 'cucina', 'bagno', 'mobili', 'lampada', 'decorazione']):
                categoria = "Casa e Giardino"
            elif any(word in nome_lower for word in ['scarpe', 'maglietta', 'jeans', 'abbigliamento', 'vestito']):
                categoria = "Abbigliamento"
            elif any(word in nome_lower for word in ['sport', 'fitness', 'palestra', 'calcio', 'tennis']):
                categoria = "Sport e Tempo Libero"
            elif any(word in nome_lower for word in ['shampoo', 'crema', 'trucco', 'profumo', 'sapone']):
                categoria = "Salute e Bellezza"
            elif any(word in nome_lower for word in ['auto', 'moto', 'pneumatico', 'olio', 'ricambio']):
                categoria = "Auto e Moto"
            else:
                categoria = "Altro"

            classificazioni[prodotto] = categoria

        return classificazioni

# Test della nuova funzione batch
print("🧪 TEST CLASSIFICAZIONE BATCH")
print("=" * 35)

prodotti_test = [
    "Play Fun BY IMC Toys Gravity Tower",
    "Scottecs Gigazine. Buon 2026 (Vol. 19)",
    "Kendal. La prima avventura",
    "iPhone 15 Pro Max"
]

classificazioni_test = classifica_prodotti_batch(prodotti_test)

for prodotto, categoria in classificazioni_test.items():
    print(f"📦 {prodotto[:30]:30} → {categoria}")

print(f"\\n✅ Classificati {len(classificazioni_test)} prodotti in una richiesta!")

In [ ]:
# CLASSIFICAZIONE EFFICIENTE DI TUTTI I PRODOTTI IN BATCH
print("🏭 CLASSIFICAZIONE AUTOMATICA IN BATCH - VERSIONE OTTIMIZZATA")
print("=" * 65)

# Ottieni lista prodotti unici (usa la colonna corretta del dataset reale)
prodotti_unici = df_vendite['Product Name'].unique()
print(f"Prodotti totali da classificare: {len(prodotti_unici)}")

# Configurazione batch ottimizzata per evitare overflow token
BATCH_SIZE = 30  # Ridotto da 20 a 15 per gestire nomi lunghi
batches = [prodotti_unici[i:i + BATCH_SIZE] for i in range(0, len(prodotti_unici), BATCH_SIZE)]

print(f"📦 Divisi in {len(batches)} batch da max {BATCH_SIZE} prodotti")
print(f"💰 Costo stimato: ~{len(batches)} richieste API invece di {len(prodotti_unici)}!")
print(f"🔧 Nomi prodotti troncati a 80 caratteri ASCII per evitare overflow")

# Classificazione batch per batch
print("\\n🤖 Classificazione in corso...")
classificazioni_complete = {}

for i, batch in enumerate(batches):
    print(f"   Batch {i+1}/{len(batches)} - Prodotti: {len(batch)}")

    # Classificare il batch corrente
    classificazioni_batch = classifica_prodotti_batch(list(batch), BATCH_SIZE)

    # Aggiungere al dizionario completo
    classificazioni_complete.update(classificazioni_batch)

    # Pausa tra le richieste per rispettare i rate limits
    if USE_REAL_API and i < len(batches) - 1:
        import time
        time.sleep(1)  # Pausa di 1 secondo tra batch

print("✅ Classificazione completata!")

# Verifica che tutti i prodotti siano stati classificati
prodotti_classificati = len(classificazioni_complete)
print(f"\\n📊 Risultato: {prodotti_classificati}/{len(prodotti_unici)} prodotti classificati")

# Mostra alcuni esempi di classificazione
print("\\n📋 ESEMPI DI CLASSIFICAZIONE:")
print("=" * 40)

esempi = list(classificazioni_complete.items())[:15]  # Prime 15
for prodotto, categoria in esempi:
    # Tronca il nome prodotto se troppo lungo per visualizzazione
    nome_display = prodotto[:45] + "..." if len(prodotto) > 45 else prodotto
    print(f"📦 {nome_display:48} → {categoria}")

if len(classificazioni_complete) > 15:
    print(f"    ... e altri {len(classificazioni_complete) - 15} prodotti")

# Statistiche finali delle categorie
from collections import Counter
categorie_count = Counter(classificazioni_complete.values())

print("\\n📈 DISTRIBUZIONE CATEGORIE:")
print("=" * 30)
for categoria, count in categorie_count.most_common():
    percentuale = (count / len(classificazioni_complete)) * 100
    print(f"  {categoria:20} → {count:3} prodotti ({percentuale:4.1f}%)")

print(f"\\n💡 Efficienza raggiunta: {len(batches)} richieste invece di {len(prodotti_unici)}")
print(f"📉 Riduzione del {((len(prodotti_unici) - len(batches)) / len(prodotti_unici) * 100):.1f}% delle richieste API!")

# 🎯 PROGETTO FINALE: ANALISI PER CATEGORIA (25 min)

Ora mettiamo tutto insieme! Combiniamo:
- ✅ **Pandas**: per manipolare i dati
- ✅ **AI**: per classificare i prodotti  
- ✅ **Matplotlib**: per visualizzare i risultati

## Obiettivo
Creare un **dashboard completo** che mostra:
1. Spesa totale per categoria
2. Andamento temporale per categoria
3. Categoria più redditizia
4. Insights per il business

In [ ]:
# STEP 1: Aggiungere le categorie al DataFrame
print("🔗 COLLEGAMENTO DATI + AI")
print("=" * 30)

# Creare una nuova colonna con le categorie AI (usando il nome corretto della colonna)
df_vendite['Categoria_AI'] = df_vendite['Product Name'].map(classificazioni_complete)
df_vendite.to_csv('ordini_classificati.csv')

for col in df_vendite.columns:
    if pd.api.types.is_datetime64tz_dtype(df_vendite[col]):
        df_vendite[col] = df_vendite[col].dt.tz_localize(None)

df_vendite.to_excel('ordini_classificati.xlsx', sheet_name='CorsoPython')

print("✅ Categorie AI aggiunte al dataset")
print("\\n📊 PRIME 10 RIGHE CON CATEGORIE:")
print(df_vendite[['Product Name', 'Total Owed', 'Categoria_AI']].head(10))

# Verifica che tutte le categorie siano assegnate
categorie_mancanti = df_vendite['Categoria_AI'].isnull().sum()
if categorie_mancanti > 0:
    print(f"⚠️  {categorie_mancanti} prodotti senza categoria")
    # Assegna "Altro" ai prodotti senza categoria
    df_vendite['Categoria_AI'].fillna('Altro', inplace=True)
    print(f"✅ Assegnata categoria 'Altro' ai prodotti mancanti")
else:
    print("✅ Tutti i prodotti hanno una categoria")

In [ ]:
# STEP 2: Analisi spesa per categoria
print("💰 ANALISI FATTURATO PER CATEGORIA")
print("=" * 40)

# Calcolare statistiche per categoria (usando i nomi corretti delle colonne)
stats_categoria = df_vendite.groupby('Categoria_AI').agg({
    'Total Owed': ['sum', 'count', 'mean'],
    'Quantity': 'sum'
}).round(2)

# Rinominare colonne
stats_categoria.columns = ['Fatturato_Totale', 'Numero_Ordini', 'Ordine_Medio', 'Quantita_Totale']

# Ordinare per fatturato
stats_categoria = stats_categoria.sort_values('Fatturato_Totale', ascending=False)

print(stats_categoria)

# Calcolare percentuali
fatturato_totale = df_vendite['Total Owed'].sum()
stats_categoria['Percentuale'] = (stats_categoria['Fatturato_Totale'] / fatturato_totale * 100).round(1)

print("\\n📊 CLASSIFICA CATEGORIE PER FATTURATO:")
for i, (categoria, row) in enumerate(stats_categoria.iterrows(), 1):
    emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "📊"
    print(f"{emoji} {i}. {categoria:20} → €{row['Fatturato_Totale']:>8,.0f} ({row['Percentuale']:4.1f}%)")

# Insights interessanti
print(f"\\n💡 INSIGHTS BUSINESS:")
categoria_top = stats_categoria.index[0]
fatturato_top = stats_categoria.iloc[0]['Fatturato_Totale']
ordini_top = stats_categoria.iloc[0]['Numero_Ordini']
print(f"🏆 Categoria più redditizia: {categoria_top}")
print(f"💰 Genera €{fatturato_top:.0f} con {ordini_top} ordini")
print(f"📈 Rappresenta il {stats_categoria.iloc[0]['Percentuale']:.1f}% del fatturato totale")

In [ ]:
# STEP 3: Visualizzazioni finali - Dashboard completo!
print("\n📈 DASHBOARD FINALE: VENDITE PER CATEGORIA")
print("=" * 50)

# Creare un dashboard con 4 grafici
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🎯 DASHBOARD VENDITE E-COMMERCE CON CLASSIFICAZIONE AI', fontsize=20, fontweight='bold', y=0.98)

# Grafico 1: Fatturato per categoria (torta)
ax1 = axes[0, 0]
colors = ['#ff9999', '#66b3ff', '#99ff99', '#ffcc99', '#ff99cc', '#c2c2f0']
stats_categoria['Fatturato_Totale'].plot(kind='pie', ax=ax1, colors=colors, autopct='%1.1f%%', startangle=90)
ax1.set_title('💰 Distribuzione Fatturato per Categoria', fontweight='bold')
ax1.set_ylabel('')

# Grafico 2: Fatturato per categoria (barre)
ax2 = axes[0, 1]
stats_categoria['Fatturato_Totale'].plot(kind='bar', ax=ax2, color='steelblue', edgecolor='black')
ax2.set_title('📊 Fatturato Totale per Categoria', fontweight='bold')
ax2.set_xlabel('Categoria')
ax2.set_ylabel('Fatturato (€)')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(axis='y', alpha=0.3)

# Grafico 3: Numero ordini per categoria
ax3 = axes[1, 0]
stats_categoria['Numero_Ordini'].plot(kind='bar', ax=ax3, color='orange', edgecolor='black')
ax3.set_title('📦 Numero Ordini per Categoria', fontweight='bold')
ax3.set_xlabel('Categoria')
ax3.set_ylabel('Numero Ordini')
ax3.tick_params(axis='x', rotation=45)
ax3.grid(axis='y', alpha=0.3)

# Grafico 4: Ordine medio per categoria
ax4 = axes[1, 1]
stats_categoria['Ordine_Medio'].plot(kind='bar', ax=ax4, color='green', edgecolor='black')
ax4.set_title('💳 Valore Ordine Medio per Categoria', fontweight='bold')
ax4.set_xlabel('Categoria')
ax4.set_ylabel('Valore Medio (€)')
ax4.tick_params(axis='x', rotation=45)
ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Dashboard completato!")

# 🎉 CONCLUSIONE CORSO PYTHON

Congratulazioni! Avete completato il corso "Python in Pratica" di FabLab Bergamo! 🎓

## 🌟 Cosa avete imparato in 4 lezioni

### Lezione 1: Fondamenti Python
- ✅ Filosofia di Python (The Zen)
- ✅ Variabili, tipi di dati, operazioni
- ✅ Strutture di controllo (if, while, for)
- ✅ Liste e dizionari

### Lezione 2: Librerie e Web
- ✅ Funzioni personalizzate
- ✅ API web con Requests
- ✅ Manipolazione immagini con Pillow
- ✅ Geometria 3D con Trimesh

### Lezione 3: Analisi Dati
- ✅ Pandas per dataset complessi
- ✅ Visualizzazioni con Matplotlib
- ✅ Statistiche e filtraggi avanzati

### Lezione 4: Intelligenza Artificiale
- ✅ Large Language Models (LLM)
- ✅ Classificazione automatica con AI
- ✅ Integrazione API OpenAI
- ✅ Dashboard completo con AI + Dati

## 🚀 I vostri prossimi passi

Ora avete le competenze per:
- 📊 **Analizzare dati** reali con Pandas
- 🤖 **Usare AI** per automatizzare compiti
- 📈 **Creare visualizzazioni** professionali
- 🌐 **Interagire con API** web
- 💡 **Sviluppare progetti** personali

## 📚 Risorse per continuare

- **Documentazione ufficiale**: [python.org](https://docs.python.org/3/)
- **Tutorial avanzati**: [Real Python](https://realpython.com/)
- **Progetti pratici**: [GitHub](https://github.com/) per ispirazione
- **Community**: [Stack Overflow](https://stackoverflow.com/) per domande

## 💭 Progetti suggeriti

1. **Analizzatore social media**: Raccogliere e analizzare dati Twitter/Instagram
2. **Chatbot intelligente**: Creare un assistente con OpenAI API
3. **Dashboard finanziario**: Visualizzare portafogli di investimento
4. **Classificatore immagini**: Usare AI per catalogare foto
5. **Web scraper**: Estrarre dati da siti web automaticamente

---

**Grazie per aver partecipato al corso!** 🙏

Continuate a programmare, sperimentare e creare cose incredibili con Python!

_FabLab Bergamo - Python in Pratica 2026_